In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/ASD_Circuits/src')
from ASD_Circuits import *

In [ ]:
BiasDF = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR")
#InfoMat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/WeightMat.Ipsi.csv", index_col=0)
InfoMat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/InfoMat.Ipsi.csv", index_col=0)
adj_mat = pd.read_csv(ConnFil, index_col=0)

In [ ]:
BiasDF.head(46)["EFFECT"].mean()

In [ ]:
# Sample N_size of strcutres as init state circuits according to their biases.
def FindInitState(tmpDF, size, BiasLim):
    STRs = tmpDF.index.values
    Biases = tmpDF["EFFECT"].values
    minBias = np.min(Biases)
    PsudoBiases = Biases - minBias + 1
    #PsudoBiases = np.power(PsudoBiases, BiasLim*150-17)
    Probs = PsudoBiases/np.sum(PsudoBiases)
    Probs[-1] = 1 - np.sum(Probs[0:-1])
    init_STRs = np.random.choice(STRs, size=size, replace=False, p=Probs)
    return init_STRs
    
    #return(Probs)
    #rand_count = 0
    #while 1:
    #    init_STRs = np.random.choice(STRs, size=size, replace=False, p=Probs)
    #    avg_bias_init_STRs = tmpDF.loc[init_STRs, "EFFECT"].mean()
    #    if avg_bias_init_STRs >= BiasLim:
    #        return init_STRs, rand_count
    #    else:
    #        #print(avg_bias_init_STRs)
    #        rand_count += 1

In [ ]:
#probs = FindInitState(BiasDF, 46, 0.37, 213)
#print(probs)

In [ ]:
%%time

minbias = 0.38
topN = 213
keepN = 46

tmpDF = BiasDF.head(topN)
STRs = tmpDF.index.values
# Determine init state
init_STRs_1 = FindInitState(tmpDF, keepN, minbias)
init_STRs_2 = FindInitState(tmpDF, keepN, minbias)


STRs = BiasDF.index.values
topN = 213
print(len(STRs))
init_state = np.zeros(topN)
for i, _STR in enumerate(STRs):
    if _STR in init_STRs_1:
        init_state[i] = 1
len(init_state)

In [ ]:
init_state

In [ ]:
ins = CircuitSearch_SA_InfoContent_v2(BiasDF, init_state, adj_mat, 
                                   InfoMat, STRs, minbias=minbias)


ins.copy_strategy = "deepcopy"
ins.Tmax=1e-1
ins.Tmin=1e-7
ins.steps=50000
Tmps, Energys, state, e = ins.anneal()
res = STRs[np.where(state==1)[0]]
score = -e


In [ ]:
plt.plot(np.arange(len(Tmps)), Energys)

In [ ]:
res = STRs[np.where(state == 1)[0]]

In [ ]:
print(tmpDF.loc[res, "EFFECT"].mean())
print(ScoreCircuit_SI_Joint(res, InfoMat))

In [ ]:
score

In [ ]:
print(RegionDistributionsList(res))

# Old

In [ ]:
g = LoadConnectome2()
EdgeWeightsDict = EdgeDict(g)

In [ ]:
# ddddd
SA_Z1 = "dat/circuit2/ASD.EW.pLI.z1.circuit.csv"
SA_Z2 = "dat/circuit2/ASD.EW.pLI.z2.circuit.csv"
SA_Z1_NoNeg = "dat/circuit2/ASD.EW.pLI.z1.NoNeg.circuits.csv"
SA_Z2_NoNeg = "dat/circuit2/ASD.EW.pLI.z2.NoNeg.circuits.csv"
SA_Z1_qn = "dat/circuit2/ASD.EW.pLI.z1.qn.circuits.csv"
SA_Z1_qn_NoNeg = "dat/circuit2/ASD.EW.pLI.z1.qn.NoNeg.circuits.csv"

In [ ]:
xx = GetBestCoheSAFil(SA_Z1_qn, EdgeWeightsDict = EdgeWeightsDict, 
                 Weighted=False, Direction=False, g=g)
print(len(xx))
RegionDistributionsList(xx)

In [ ]:
xx = GetBestCoheSAFil(SA_Z1_qn_NoNeg, EdgeWeightsDict = EdgeWeightsDict, 
                 Weighted=False, Direction=False, g=g)
print(len(xx))
RegionDistributionsList(xx)

In [ ]:
xx = GetBestCoheSAFil(SA_Z1, EdgeWeightsDict = EdgeWeightsDict, 
                 Weighted=False, Direction=False, g=g)
print(len(xx))
RegionDistributionsList(xx)

In [ ]:
xx = GetBestCoheSAFil(SA_Z2, EdgeWeightsDict = EdgeWeightsDict, 
                 Weighted=False, Direction=False, g=g)
print(len(xx))
RegionDistributionsList(xx)

In [ ]:
xx = list(xx)
xx2 = xx + ["Dentate_gyrus", "Field_CA1", "Field_CA3", "Field_CA2"]
score, x = ScoreSTRSet(g, xx2, {})
score

In [ ]:
xx = list(xx)
xx2 = xx + ["Dentate_gyrus", "Field_CA1", "Field_CA3", "Field_CA2", "Subiculum_dorsal_part"]
score, x = ScoreSTRSet(g, xx2, {})
score

In [ ]:
xx = GetBestCoheSAFil(SA_Z1_NoNeg, EdgeWeightsDict = EdgeWeightsDict, 
                 Weighted=False, Direction=False, g=g)
print(len(xx))
RegionDistributionsList(xx)

In [ ]:
xx = GetBestCoheSAFil(SA_Z2_NoNeg, EdgeWeightsDict = EdgeWeightsDict, 
                 Weighted=False, Direction=False, g=g)
print(len(xx))
RegionDistributionsList(xx)

In [ ]:
#Spark_EWS = pd.read_csv("dat/bias2/ASD.EW.pLI.z2.csv", index_col="STR")
Spark_EWS = pd.read_csv("dat/bias2/ASD.159.pLI.z2.csv", index_col="STR")
#Spark_EWS = pd.read_csv("dat/bias2/ASD.EW.pLi.Spec.bias.csv", index_col="STR")
#Spark_EWS = pd.read_csv("dat/bias2/ASD.EW.pLi.NonNeg.Spec.bias.csv", index_col="STR")
#Spark_EWS = pd.read_csv("dat/bias2/ASD.EW.NonNeg.avgZ.bias.csv", index_col="STR")
#Spark_EWS = pd.read_csv("dat/Jon_data/exprank-for_jw.csv", index_col="STR")

In [ ]:
states = run_CircuitOpt(g, EdgeWeightsDict, Spark_EWS, False, False)

In [ ]:
CandidateNodes = Spark_EWS.head(50).index.values
Cir_STRs = CandidateNodes[np.where(states==1)[0]]
ScoreSTRSet(g, Cir_STRs, {})

In [ ]:
print(len(Cir_STRs))

In [ ]:
xx = RegionDistributions(Spark_EWS, show=True, states=states)

In [ ]:
Cir_STRs = list(CandidateNodes[np.where(states==1)[0]]) + ["Dentate_gyrus", "Field_CA1", 
                                                           "Field_CA2", "Field_CA3"]
ScoreSTRSet(g, Cir_STRs, {})

In [ ]:
Cir_STRs = list(CandidateNodes[np.where(states==1)[0]]) + ["Paramedian_lobule"]
ScoreSTRSet(g, Cir_STRs, {})

In [ ]:
# 0.2754

In [ ]:
# 0.360

In [ ]:
sim1 = pd.read_csv("/Users/jiayao/Work/ASD_Circuits/src/xxx.50.csv")
sim1.head(2)

In [ ]:
candidates = sim1.columns.values
Init_States = np.ones(len(candidates))
ins = MostCohesiveCirtuis(Init_States, g, candidates, EdgeWeightsDict)
Cohes = []
for i, row in sim1.iterrows():
    ins.state = row.values
    #print(ins.state)
    InCirtuitNodes = ins.CandidateNodes[np.where(ins.state==1)[0]]
    Cohes.append(1-ins.energy())

In [ ]:
(2 ** 7.8) / (2 ** 7.4)

In [ ]:
(2 ** 1.6159985461036568) / (2 ** 1.3691164748505738)

In [ ]:
best_id = np.argmax(Cohes)
state = sim1.loc[1, :].values
print(Cohes[best_id])

In [ ]:
InCirtuitNodes = ins.CandidateNodes[np.where(state==1)[0]]
InCirtuitNodes = np.append(InCirtuitNodes, ["Parafascicular_nucleus", "Caudoputamen"])
print(len(InCirtuitNodes))
xx = RegionDistributionsList(InCirtuitNodes)

# Cross_Region Connections

In [ ]:
# Load Exp Bias
SparkMetaBias = pd.read_csv("dat/Jon_data/Spark_Final_ExpBias.csv")
SparkMetaBias.columns = ["STR", "EFFECT", "Rank", "NGene"]
SparkMetaBias = SparkMetaBias.set_index("STR")

adj_mat = pd.read_csv(ConnFil, index_col=0)

In [ ]:
adj_mat = pd.read_csv(ConnFil, index_col=0)
adj_mat_local, adj_mat_distal = LocalDistal_Region()
g = LoadConnectome2()
EdgeWeightsDict = EdgeDict(g)
g_local_region = LoadConnectome2(ConnFil=adj_mat_local)
g_distal_region = LoadConnectome2(ConnFil=adj_mat_distal)
EdgeWeightsDict_local_region = EdgeDict(g_local_region, keyon="label")
EdgeWeightsDict_distal_region = EdgeDict(g_distal_region, keyon="label")

In [ ]:
state_distal = run_CircuitOpt(g_distal_region, EdgeWeightsDict_distal_region, SparkMetaBias, False, False)

In [ ]:
xx = RegionDistributions(SparkMetaBias, show=True, states=state_distal)

In [ ]:
graph_mats_cartesians = []
EdgeWeights_cartesians = []
for i in range(10):
    xx = pd.read_csv("../dat/allen-mouse-conn/cartesian_deciles_limit_{}.csv".format(i), index_col=0)
    g_ = LoadConnectome2(xx)
    graph_mats_cartesians.append(g_)
    EdgeWeightsDict_ = EdgeDict(g_, keyon="label")
    EdgeWeights_cartesians.append(EdgeWeightsDict_)

In [ ]:
States = []
for i in range(10):
    state_ = run_CircuitOpt(graph_mats_cartesians[i], EdgeWeights_cartesians[i], SparkMetaBias, False, False)
    print(i)
    xx = RegionDistributions(SparkMetaBias, show=True, states=state_)
    States.append(state_)
    print()

In [ ]:
xx = pd.read_csv("../dat/allen-mouse-conn/cartesian_deciles_limit_{}.csv".format(9), index_col=0)

In [ ]:
xx.loc["Anteromedial_visual_area", :].sort_values(ascending=False)

In [ ]:
xx.loc[:, "Anteromedial_visual_area"].sort_values(ascending=False)

In [ ]:
Cartesian_distances = pd.read_csv("../dat/allen-mouse-conn/Dist_CartesianDistance.csv",
                                   index_col=0)

In [ ]:
Cartesian_distances.loc["Anteromedial_visual_area","Orbital_area_ventrolateral_part"]

In [ ]:
ASD_Meta_avgZ = pd.read_csv("dat/bias2/ASD.Meta.avgZ.bias.csv", index_col="STR")

In [ ]:
ASD_Meta_avgZ.head(2)

In [ ]:
ComareDF = CompareSTROverlap(SparkMetaBias, ASD_Meta_avgZ, "ZofZ", "Z")

In [ ]:
print(ComareDF["STR.only.in.ZofZ"].values)
print(ComareDF["STR.only.in.Z"].values)

In [ ]:
state_z = run_CircuitOpt(g, EdgeWeightsDict, ASD_Meta_avgZ, False, False)

In [ ]:
CandidateNodes = ASD_Meta_avgZ.head(50).index.values
Cir_STRs = CandidateNodes[np.where(state_z==1)[0]]

In [ ]:
SA_wd = "/Users/jiayao/Work/ASD_Circuits/src/dat/circuits/ASD_Pad1_SA.csv"
weighted = False; directed = False
Incircuit_ = GetBestCoheSAFil(SA_wd, weighted, directed, g=g, EdgeWeightsDict=EdgeWeightsDict)

In [ ]:
print(set(Incircuit_).difference(set(Cir_STRs)))
print(set(Cir_STRs).difference(set(Incircuit_)))

In [ ]:
i,j = np.random.choice(213, 2)
print(i, j)

In [ ]:
def run_CircuitOpt2(g, EdgeWeightsDictw, init_STRs, weighted, directed):
    CandidateNodes = np.array(g.vs["label"]) # all nodes are candidate
    Init_states = np.zeros(len(CandidateNodes))
    for idx in range(len(CandidateNodes)):
        if CandidateNodes[idx] in init_STRs:
            Init_states[idx] = 1
    #print(CandidateNodes)
    ins = MostCohesiveCirtuisGivenNCandidates(Init_states, g, CandidateNodes, EdgeWeightsDict, 
                              Weighted=weighted, Direction=directed)
    ins.copy_strategy = "deepcopy"
    ins.Tmax = 0.1
    ins.Tmin = 0.00001
    Tmps, Energys, state, e = ins.anneal()
    print("Done")
    return state

In [ ]:
state = np.array([0,1,1,0,0,1])
np.where(state==1)

In [ ]:
xx = run_CircuitOpt2(g, EdgeWeightsDict, Incircuit_, False, False)

In [ ]:
InCirtuitNodes = np.array(g.vs["label"])[np.where(xx==1)[0]]

In [ ]:
RegionDistributionsList(InCirtuitNodes)

In [ ]:
np.mean(ScoreSTRSet(g, InCirtuitNodes, EdgeWeightsDict,  False, False))

In [ ]:
states_84 = run_CircuitOpt(g, EdgeWeightsDict, SparkMetaBias, False, False, topN=84)

In [ ]:
xx = SparkMetaBias.index.values[np.where(states_84==1)[0]]

In [ ]:
print(len(xx))

In [ ]:
RegionDistributionsList(xx)

In [ ]:
SparkMetaBias

# New_Neibor Bias Cohesivness

In [ ]:
SA_ = "dat/circuits/ASD_Pad1_SA.csv"
Weighted = False; Directed = False
Incircuit_ = GetBestCoheSAFil(SA_, Weighted, Directed, g=g, EdgeWeightsDict=EdgeWeightsDict)

In [ ]:
Incircuit_
xx_xxs = []
for x in g.vs["label"]:
    if x in Incircuit_:
        xx_xxs.append(1)
    else:
        xx_xxs.append(0)

In [ ]:
#annotate with bias
str2reg = STR2Region()
for i in enumerate(g.vs):
    label_ = g.vs[i]["label"][0]
    g.vs[i]["region"] = str2reg[label_]
    g.vs[i]["Bias"] = SparkMetaBias.loc[label_, "EFFECT"]
    g.vs[i]["Bias_Rank"] = SparkMetaBias.loc[label_, "Rank"]

##### Start rerun here

In [ ]:
xx = run_CircuitOpt_neighborBias(g, EdgeWeightsDict, False, False)

In [ ]:
print(NeiborBias(g, xx_xxs))
yy = np.array((g.vs["label"]))[np.where(xx==1)[0]]
print(yy)
print(NeiborBias(g, xx))

In [ ]:
RegionDistributionsList(yy)

In [ ]:
NeiborBias(g, xx, verbose=True)

In [ ]:
SparkMetaBias.loc[yy, :]["EFFECT"].hist()

In [ ]:
SparkMetaBias.loc[yy, :].sort_values("EFFECT").head(5)

In [ ]:
bad_strs = SparkMetaBias.loc[yy, :].sort_values("EFFECT").index[:5].values
bad_strs

##### Manipulate STRs (add drop)

In [ ]:
def change_str(STRs, Actions, g, cur_state):
    new_state = cur_state.copy()
    for STR, action in zip(STRs, Actions):
        idx_str = g.vs.select(label_eq=STR)[0].index
        if action == "+":
            new_state[idx_str] = 1
        else:
            new_state[idx_str] = 0
    return new_state

xx_new = change_str(bad_strs, ["-"]*len(bad_strs), g, xx)
yy_new = np.array((g.vs["label"]))[np.where(xx_new==1)[0]]
print(NeiborBias(g, xx_new, verbose=True))

In [ ]:
print(NeiborBias(g, xx_xxs, verbose=True))

In [ ]:
xx2 = run_CircuitOpt_neighborBias(g, EdgeWeightsDict, False, False)

In [ ]:
print(NeiborBias(g, xx_xxs))
yy = np.array((g.vs["label"]))[np.where(xx2==1)[0]]
print(yy)
print(NeiborBias(g, xx2, verbose=True))

In [ ]:
RegionDistributionsList(yy)

In [ ]:
SparkMetaBias.loc[yy, :]["EFFECT"].hist()

In [ ]:
#SparkMetaBias.head(50) == 0.230964

In [ ]:
SparkMetaBias

In [ ]:
xx3 = run_CircuitOpt_neighborBias(g, EdgeWeightsDict, False, False)

In [ ]:
print(NeiborBias(g, xx_xxs))
yy = np.array((g.vs["label"]))[np.where(xx3==1)[0]]
print(yy)
print(NeiborBias(g, xx3, verbose=True))

In [ ]:
print(len(yy))
RegionDistributionsList(yy)

In [ ]:
xx4 = run_CircuitOpt_neighborBias(g, EdgeWeightsDict, False, False)

In [ ]:
print(NeiborBias(g, xx_xxs))
yy = np.array((g.vs["label"]))[np.where(xx4==1)[0]]
print(yy)
print(NeiborBias(g, xx4, verbose=True))

In [ ]:
print(len(yy))
RegionDistributionsList(yy)

In [ ]:
(6 * 0.3 + 0.2 ) / 7

# Old

In [ ]:
sim1 = pd.read_csv("/Users/jiayao/Work/ASD_Circuits/src/dat/circuits/ASD_Pad1_W_D_SA.csv")
sim1.head(2)


In [ ]:
g = LoadConnectome2(ConnFil=
                    "../dat/allen-mouse-conn/norm_density-max_ipsi_contra-pval_0.05-deg_min_1-by_weight_pvalue.csv")
EdgeWeightsDict = EdgeDict(g, keyon="label")

In [ ]:
sim1 = pd.read_csv("/Users/jiayao/Work/ASD_Circuits/src/dat/circuits/ASD_Pad1_W_D_SA.csv")
sim1.head(2)

In [ ]:
candidates = sim1.columns.values
Init_States = np.ones(len(candidates))
ins = MostCohesiveCirtuis(Init_States, g, candidates, EdgeWeightsDict)
Cohes = []
for i, row in sim1.iterrows():
    ins.state = row.values
    #print(ins.state)
    InCirtuitNodes = ins.CandidateNodes[np.where(ins.state==1)[0]]
    Cohes.append(1-ins.energy())

In [ ]:
best_id = np.argmax(Cohes)
state = sim1.loc[1, :].values
print(Cohes[best_id])

In [ ]:
InCirtuitNodes = ins.CandidateNodes[np.where(state==1)[0]]
InCirtuitNodes = np.append(InCirtuitNodes, ["Parafascicular_nucleus", "Caudoputamen"])
print(len(InCirtuitNodes))
print(list(InCirtuitNodes))

In [ ]:
OutCirtuitNodes = ins.CandidateNodes[np.where(state==0)[0]]
print(len(OutCirtuitNodes))
print(list(OutCirtuitNodes))

In [ ]:
Subiculum_ventral_part
Parasubiculum

In [ ]:
color_dict = {"Isocortex":"red", "Thalamus":"blue", "Striatum":"green", "Hippocampal_formation":"yellow", 
              "Cortical_subplate":"pink", "Hippocampal_region":"orange", "Olfactory_areas":"purple",}

In [ ]:
coord_dict = {}
top_nodes = g.vs.select(label_in=InCirtuitNodes)
g2 = g.copy().subgraph(top_nodes)
for Node in InCirtuitNodes:
    Cohe, ITR = CohesivenessSingleNodeMaxInOut(g, g2, Node, EdgeWeightsDict)
    coord_dict[Node] = (Cohe, np.log10(min(10, ITR)))
STR2REG, REG2STR = LoadSTR2REG()
Regions = list(REG2STR.keys())
plt.style.use('default')
fig, ax = plt.subplots(dpi=300, figsize=(16,12), frameon=False)
y_poss = []
x_poss = []
for K,v in coord_dict.items():
    x,y = v
    ax.scatter(x,y, s=160, c=color_dict[STR2REG[K]], marker="o", alpha=0.5)#, label=STR2REG[K])

    if x>0 and x<1 and y>0.9 and y<1:
        print(K)
        
    if K == "Retrosplenial_area_lateral_agranular_part":
        y -= 0.03
    if K == "Lateral_septal_nucleus_caudal_caudodorsal_part":
        y += 0.03
    if K == "Subiculum_dorsal_part":
        y -= 0.03
    if K == "Claustrum":
        y -= 0.03
    if K == "Dorsal_peduncular_area":
        y += 0.03
    if K == "Dentate_gyrus":
        y -= 0.03
    if K == "Parafascicular_nucleus":
        y -= 0.03
        
    y_poss.append(y)
    x_poss.append(x)
    ax.text(x,y, s=K, fontstyle="normal", fontsize=10, fontweight="semibold")
"""
factor = 0.98
for e in g2.es:
    point1 = coord_dict[g2.vs[e.source]["label"]]
    point2 = coord_dict[g2.vs[e.target]["label"]]
    plt.arrow(point1[0], point1[1], 
              dx=(point2[0]-point1[0]) * factor, dy=(point2[1]-point1[1]) * factor, 
              head_width = 0.01, width = 0.00001, ec="grey", alpha=0.6)
"""
plt.xlabel("Cohesiveness", fontsize=20)
plt.ylabel("ITR", fontsize=20)

#y_labels = [np.power(item, 10) for item in ax.get_yticklabels()]
#y_labels = [item for item in ax.get_yticklabels()]
#print(y_labels)
#ax.set_yticklabels(y_labels)

ax.set_yticks([-1,0,1])
ax.set_yticklabels(['Integrator 0', "1", "Broadcaster Inf"], fontsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis="x", labelsize=20)

markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in color_dict.values()]
plt.legend(markers, color_dict.keys(), numpoints=1, fontsize=15)
#plt.show()
plt.title("")
plt.savefig('Circuit.WD.pdf')  

In [ ]:
coord_dict = {}
top_nodes = g.vs.select(label_in=InCirtuitNodes)
g2 = g.copy().subgraph(top_nodes)
for Node in InCirtuitNodes:
    Cohe, ITR = CohesivenessSingleNodeMaxInOut(g, g2, Node, EdgeWeightsDict, weighted=False)
    coord_dict[Node] = (Cohe, np.log10(min(10, ITR)))
STR2REG, REG2STR = LoadSTR2REG()
Regions = list(REG2STR.keys())
plt.style.use('default')
fig, ax = plt.subplots(dpi=300, figsize=(16,12), frameon=False)
y_poss = []
x_poss = []
for K,v in coord_dict.items():
    x,y = v
    ax.scatter(x,y, s=160, c=color_dict[STR2REG[K]], marker="o", alpha=0.5)#, label=STR2REG[K])

    if x>0 and x<1 and y>0.9 and y<1:
        print(K)
        
    if K == "Retrosplenial_area_lateral_agranular_part":
        y -= 0.03
    if K == "Lateral_septal_nucleus_caudal_caudodorsal_part":
        y += 0.03
    if K == "Subiculum_dorsal_part":
        y -= 0.03
    if K == "Claustrum":
        y -= 0.03
    if K == "Dorsal_peduncular_area":
        y += 0.03
    if K == "Dentate_gyrus":
        y -= 0.03
    if K == "Parafascicular_nucleus":
        y -= 0.03
        
    y_poss.append(y)
    x_poss.append(x)
    ax.text(x,y, s=K, fontstyle="normal", fontsize=10, fontweight="semibold")
"""
factor = 0.98
for e in g2.es:
    point1 = coord_dict[g2.vs[e.source]["label"]]
    point2 = coord_dict[g2.vs[e.target]["label"]]
    plt.arrow(point1[0], point1[1], 
              dx=(point2[0]-point1[0]) * factor, dy=(point2[1]-point1[1]) * factor, 
              head_width = 0.01, width = 0.00001, ec="grey", alpha=0.6)
"""
plt.xlabel("Cohesiveness", fontsize=20)
plt.ylabel("ITR", fontsize=20)

#y_labels = [np.power(item, 10) for item in ax.get_yticklabels()]
#y_labels = [item for item in ax.get_yticklabels()]
#print(y_labels)
#ax.set_yticklabels(y_labels)

ax.set_yticks([-1,0,1])
ax.set_yticklabels(['Intergrator 0', "1", "Broadcastor Inf"], fontsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis="x", labelsize=20)

markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in color_dict.values()]
plt.legend(markers, color_dict.keys(), numpoints=1, fontsize=15)
plt.show()
#plt.savefig('Circuit.WD.pdf')  

In [ ]:
np.power(1, 10)

In [ ]:
np.log10(10)